<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/12_29_sji_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 55 kB/s 


In [2]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
import tensorflow as tf
import keras
from numpy import array
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib as mpl
from IPython.core.display import display, HTML
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, f1_score
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
import urllib.request
import json
import datetime
import os
from datetime import datetime
from tqdm import tqdm
import warnings
from glob import glob
from catboost import CatBoostClassifier

## 드라이브마운트

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('/content/gdrive/MyDrive/data/know/train/*.csv'))]

In [5]:
know_train[0].head(2)

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,aq5_2,aq6_1,aq6_2,aq7_1,aq7_2,aq8_1,aq8_2,aq9_1,aq9_2,aq10_1,aq10_2,aq11_1,aq11_2,aq12_1,aq12_2,aq13_1,aq13_2,aq14_1,aq14_2,aq15_1,aq15_2,aq16_1,aq16_2,aq17_1,aq17_2,aq18_1,aq18_2,aq19_1,aq19_2,aq20_1,...,bq18_4,bq18_5,bq18_6,bq18_7,bq19,bq19_1,bq20,bq21,bq22,bq23,bq24_1,bq24_2,bq24_3,bq24_4,bq24_5,bq24_6,bq24_7,bq24_8,bq25,bq26,bq27,bq28,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode
0,0,3,3,3,3,3,3,4,4,3,4,3,3,2,2,2,2,2,2,3,3,3,3,4,4,3,3,3,2,3,3,5,6,5,6,4,5,2,2,5,...,3,2,3,3,2,생산설비의 자동화로,2,3,5,10.0,1,1,1,2,2,2,2,2,3,3,4,3,2,없다,없다,없다,건설현장 노무직,없다,4,1,52,2,실업,1,1,1,4000,,2200,825101
1,1,4,5,4,5,3,4,3,4,3,4,1,,1,,1,,1,,2,3,3,4,3,4,4,5,2,2,3,4,3,4,3,4,3,4,1,,2,...,4,2,2,4,2,건설 수주가 없어서,3,3,6,50.0,2,2,2,2,2,2,2,2,4,3,4,3,2,없다,"매타기, 드라이버, 가위, CAD",,,,2,1,38,4,건축공학,1,1,1,,,2400,140204


In [6]:
train_2017 = know_train[0]
X_2017 = train_2017.iloc[:,1:-1]
y_2017 = train_2017.iloc[:,-1]

# 텍스트 열 삭제
X_2017.drop(['bq19_1', 'bq30',	'bq31',	'bq32',	'bq33',	'bq34','bq38_1'],axis = 1, inplace = True)
print(X_2017)

X_train_2017,X_test_2017,y_train_2017,y_test_2017 = train_test_split(X_2017,y_2017,test_size=0.2,random_state=10)

      aq1_1 aq1_2  aq2_1 aq2_2  aq3_1  ... bq39_2  bq40 bq41_1  bq41_2 bq41_3
0         3     3      3     3      3  ...      1     1   4000           2200
1         4     5      4     5      3  ...      1     1                  2400
2         3     4      3     4      3  ...      1     1   4000           2400
3         3     3      3     3      3  ...      1     1   7000           3500
4         4     5      3     4      3  ...      1     1   4000           2500
...     ...   ...    ...   ...    ...  ...    ...   ...    ...     ...    ...
9481      3     5      2     4      3  ...      1     1   5200           1800
9482      5     5      5     5      5  ...      1     1   4000           3000
9483      3     3      4     6      3  ...      4     2   2700           1500
9484      3     5      3     5      4  ...      1     1   6800           2500
9485      3     4      3     4      3  ...      1     1   4000           3700

[9486 rows x 147 columns]


In [7]:
cat_features = list()
for i in X_train_2017.columns:
    cat_features.append(i)
print(cat_features)

del_features = ['bq23', 'bq37', 'bq41_1', 'bq41_2', 'bq41_3']
for i in del_features:
  cat_features.remove(i)
print(cat_features)


['aq1_1', 'aq1_2', 'aq2_1', 'aq2_2', 'aq3_1', 'aq3_2', 'aq4_1', 'aq4_2', 'aq5_1', 'aq5_2', 'aq6_1', 'aq6_2', 'aq7_1', 'aq7_2', 'aq8_1', 'aq8_2', 'aq9_1', 'aq9_2', 'aq10_1', 'aq10_2', 'aq11_1', 'aq11_2', 'aq12_1', 'aq12_2', 'aq13_1', 'aq13_2', 'aq14_1', 'aq14_2', 'aq15_1', 'aq15_2', 'aq16_1', 'aq16_2', 'aq17_1', 'aq17_2', 'aq18_1', 'aq18_2', 'aq19_1', 'aq19_2', 'aq20_1', 'aq20_2', 'aq21_1', 'aq21_2', 'aq22_1', 'aq22_2', 'aq23_1', 'aq23_2', 'aq24_1', 'aq24_2', 'aq25_1', 'aq25_2', 'aq26_1', 'aq26_2', 'aq27_1', 'aq27_2', 'aq28_1', 'aq28_2', 'aq29_1', 'aq29_2', 'aq30_1', 'aq30_2', 'aq31_1', 'aq31_2', 'aq32_1', 'aq32_2', 'aq33_1', 'aq33_2', 'aq34_1', 'aq34_2', 'aq35_1', 'aq35_2', 'aq36_1', 'aq36_2', 'aq37_1', 'aq37_2', 'aq38_1', 'aq38_2', 'aq39_1', 'aq39_2', 'aq40_1', 'aq40_2', 'aq41_1', 'aq41_2', 'bq1', 'bq2', 'bq3', 'bq4', 'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5', 'bq5_1', 'bq5_2', 'bq6', 'bq7', 'bq8_1', 'bq8_2', 'bq8_3', 'bq9', 'bq10', 'bq11', 'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5', 

In [ ]:
model = CatBoostClassifier(task_type='GPU')

model.fit(X_train_2017, y_train_2017, cat_features, verbose=True)
# Get predicted classes
preds_class = model.predict(X_test_2017)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(X_test_2017)
# Get predicted RawFormulaVal
preds_raw = model.predict(X_test_2017, prediction_type='RawFormulaVal')

In [2]:
model.save_model('/content/gdrive/MyDrive/data/know',
           format="python",
           export_parameters=None,
           pool=None)
# 아니면 경로만 지정하고

NameError: ignored